In [1]:
from langgraph.graph import Graph
from pydantic import BaseModel
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
llm=ChatGroq(model="llama3-groq-70b-8192-tool-use-preview")

In [2]:
@tool
def add_tools(a:int,b:int):
    '''add a and b'''
    return a+b

In [3]:
@tool
def subtract(c:int,b:int):
    '''subtract c and d'''
    return c-b

In [4]:
@tool
def multiple(d:int,e:int):
    '''multiple d and e'''
    return d * e

In [5]:
tools=[add_tools,subtract]
llmwithtools=llm.bind_tools(tools)
print(llmwithtools)
messages=["hii i want to add 5 and 7"]

bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023DCBF8CD10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023DCBF8D880>, model_name='llama3-groq-70b-8192-tool-use-preview', model_kwargs={}, groq_api_key=SecretStr('**********')) kwargs={'tools': [{'type': 'function', 'function': {'name': 'add_tools', 'description': 'add a and b', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'subtract', 'description': 'subtract c and d', 'parameters': {'properties': {'c': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['c', 'b'], 'type': 'object'}}}]} config={} config_factories=[]


In [6]:
aimessage=llmwithtools.invoke(messages)

In [11]:
print(aimessage)

content='' additional_kwargs={'tool_calls': [{'id': 'call_fwv3', 'function': {'arguments': '{"a": 5, "b": 7}', 'name': 'add_tools'}, 'type': 'function'}, {'id': 'call_7kr0', 'function': {'arguments': '{"a": 10, "b": 15}', 'name': 'add_tools'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 291, 'total_tokens': 355, 'completion_time': 0.201384093, 'prompt_time': 0.020482976, 'queue_time': 0.012829122999999998, 'total_time': 0.221867069}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-e704c774-1fe8-46dc-94c7-1e24f30cd195-0' tool_calls=[{'name': 'add_tools', 'args': {'a': 5, 'b': 7}, 'id': 'call_fwv3', 'type': 'tool_call'}, {'name': 'add_tools', 'args': {'a': 10, 'b': 15}, 'id': 'call_7kr0', 'type': 'tool_call'}] usage_metadata={'input_tokens': 291, 'output_tokens': 64, 'total_tokens': 355}


In [12]:
for tool_call in aimessage.tool_calls:
    print(tool_call["name"].lower())
    selectedtool={"add_tools":add_tools,"subtract":subtract}[tool_call["name"].lower()]
    print(selectedtool)
    toolmessage=selectedtool.invoke(tool_call)
    print(toolmessage)
    messages.append(toolmessage)
print(messages)

add_tools
name='add_tools' description='add a and b' args_schema=<class 'langchain_core.utils.pydantic.add_tools'> func=<function add_tools at 0x0000023DCBF87A60>
content='12' name='add_tools' tool_call_id='call_fwv3'
add_tools
name='add_tools' description='add a and b' args_schema=<class 'langchain_core.utils.pydantic.add_tools'> func=<function add_tools at 0x0000023DCBF87A60>
content='25' name='add_tools' tool_call_id='call_7kr0'
['hii i want to add 5 and 7', ToolMessage(content='12', name='add_tools', tool_call_id='call_fwv3'), ToolMessage(content='25', name='add_tools', tool_call_id='call_7kr0')]


In [31]:
response=llmwithtools.invoke(messages)

In [32]:
response.content

'The sum of 5 and 7 is 12. Do you need help with anything else?'

In [ ]:
class input(BaseModel):
    answer:int
    question:str
    

In [35]:
@tool
def multiply3(a:int,b:int,c:int):
    '''multiply a,b and c'''
    return a*b*c

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
Assistant_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a mathematics expert"
            "use the provided tools to perform multiply add and subtract operations"

        ),
        ("placeholder","{question}"),
    ]
)
safe_tools